In [ ]:
library(SciServer)
library(httr)
library(jsonlite)
library(utils)

# Define login Name and password before running these examples
Authentication_loginName = '***';
Authentication_loginPassword = '***'

Authentication_login_sharedWithName = '***'
Authentication_login_sharedWithPassword = '***'

# Authentication section

In [ ]:
#?Authentication.login

In [ ]:
#logging in and getting current token from different ways

token1 = Authentication.login(Authentication_loginName, Authentication_loginPassword);
token2 = Authentication.getToken()
token4 = Authentication.token
print(paste("token1=", token1))
print(paste("token2=", token2))
print(paste("token4=", token4))

In [ ]:
#getting curent user info

user = Authentication.getKeystoneUserWithToken(token1)
print(paste("userName=", user$userName))
print(paste("id=", user$id))

In [ ]:
#reseting the current token to another value:

Authentication.setToken("myToken1")
token5 = Authentication.getToken()
print(paste("token5=", token5))

In [ ]:
#logging-in again

token1 = Authentication.login(Authentication_loginName, Authentication_loginPassword);
print(token1)

# LoginPortal section:

In [ ]:
#?LoginPortal.login

In [ ]:
#logging in and getting current token from different ways

token1 = LoginPortal.login(Authentication_loginName, Authentication_loginPassword);
token2 = LoginPortal.getToken()
print(paste("token1=", token1))
print(paste("token2=", token2))

In [ ]:
#getting curent user info

user = LoginPortal.getKeystoneUserWithToken(token1)
print(paste("userName=", user$user))
print(paste("id=", user$id))
user

In [ ]:
#reseting the current token to another value:

LoginPortal.setToken("myToken2")
token6 = LoginPortal.getToken()
print(paste("token6=", token6))

In [ ]:
#logging-in again

token1 = LoginPortal.login(Authentication_loginName, Authentication_loginPassword);
print(token1)

# CasJobs section:

In [ ]:
#?CasJobs.executeQuery

In [ ]:
#Defining databse context and query, and other variables

CasJobs_TestDatabase = "MyDB"
CasJobs_TestQuery = "select 4 as Column1, 5 as Column2 "
CasJobs_TestTableName1 = "MyNewtable1"
CasJobs_TestTableName2 = "MyNewtable2"
CasJobs_TestTableCSV = "Column1,Column2\n4,5\n"
CasJobs_TestFitsFile = "SciScriptTestFile.fits"
CasJobs_TestCSVFile = "SciScriptTestFile.csv"

In [ ]:
#get user schema info

casJobsId = CasJobs.getSchemaName()
print(casJobsId)

In [ ]:
#get info about tables inside MyDB database context:

tables = CasJobs.getTables(context="MyDB")
print(tables)

In [ ]:
#execute a quick SQL query:

df = CasJobs.executeQuery(sql=CasJobs_TestQuery, context=CasJobs_TestDatabase, format="dataframe")
df

In [ ]:
#submit a job, which inserts the query results into a table in the MyDB database context. 
#Wait until the job is done and get its status.

jobId = CasJobs.submitJob(sql=paste(CasJobs_TestQuery, " into MyDB.", CasJobs_TestTableName1, sep=""), context="MyDB")
jobDescription = CasJobs.waitForJob(jobId=jobId, verbose=FALSE)
print(jobId)
print(jobDescription)

In [ ]:
# drop or delete table in MyDB database context

df = CasJobs.executeQuery(sql=paste("DROP TABLE ", CasJobs_TestTableName1, sep=""), context="MyDB", format="dataframe")
print(df)

In [ ]:
#get job status

jobId = CasJobs.submitJob(sql=CasJobs_TestQuery, context=CasJobs_TestDatabase)
jobDescription = CasJobs.getJobStatus(jobId)
print(jobId)
print(jobDescription)

In [ ]:
#cancel a job

jobId = CasJobs.submitJob(sql=CasJobs_TestQuery, context=CasJobs_TestDatabase)
jobDescription = CasJobs.cancelJob(jobId=jobId)
print(jobId)
print(jobDescription)

In [ ]:
#execute a query and write a local Fits file containing the query results:

result = CasJobs.writeFitsFileFromQuery(fileName=CasJobs_TestFitsFile, queryString=CasJobs_TestQuery, context="MyDB")
print(result)

In [ ]:
#delete local FITS file just created:

file.remove(CasJobs_TestFitsFile)

In [ ]:
#uploads a dataframe into a Database table

df = CasJobs.executeQuery(sql=CasJobs_TestQuery, context=CasJobs_TestDatabase, format="dataframe")
response = CasJobs.uploadDataFrameToTable(df, tableName=CasJobs_TestTableName2, context="MyDB")
table = CasJobs.executeQuery(sql=paste("select * from ", CasJobs_TestTableName2, sep=""), context="MyDB", format="dataframe")
print(response)
print(table)

In [ ]:
# drop or delete table just created:

result2 = CasJobs.executeQuery(sql=paste("DROP TABLE ", CasJobs_TestTableName2, sep=""), context="MyDB", format="dataframe")
print(result2)

In [ ]:
#upload csv data string into a database table:

result3 = CasJobs.uploadCSVDataToTable(csv=CasJobs_TestTableCSV, tableName=CasJobs_TestTableName2, context="MyDB")
df2 = CasJobs.executeQuery(sql=paste("select * from ", CasJobs_TestTableName2, sep=""), context="MyDB", format="dataframe")
print(result3)
print(df2)

In [ ]:
# drop or delete table just created:

result4 = CasJobs.executeQuery(sql=paste("DROP TABLE ", CasJobs_TestTableName2, sep=""), context="MyDB", format="dataframe")
print(result4)

#  SkyServer section:

In [ ]:
#SkyServer.sqlSearch

In [ ]:
#defining sql query and SDSS data relelease:

SkyServer_TestQuery = "select top 1 specobjid, ra, dec from specobj order by specobjid"
SkyServer_DataRelease = "DR13"

In [ ]:
#Exectute sql query

df = SkyServer.sqlSearch(sql=SkyServer_TestQuery, dataRelease=SkyServer_DataRelease)
print(df)

In [ ]:
#get an image cutout

img = SkyServer.getJpegImgCutout(ra=197.614455642896, dec=18.438168853724, width=512, height=512, scale=0.4, 
                                 dataRelease=SkyServer_DataRelease,opt="OG",
                                 query="SELECT TOP 100 p.objID, p.ra, p.dec, p.r FROM fGetObjFromRectEq(197.6,18.4,197.7,18.5) n, PhotoPrimary p WHERE n.objID=p.objID")
plot(0:1, 0:1, type = "n")
rasterImage(img, 0, 0, 1, 1)

In [ ]:
# do a radial search of objects:

df = SkyServer.radialSearch(ra=258.25, dec=64.05, radius=0.1, dataRelease=SkyServer_DataRelease)
print(df)

In [ ]:
#do rectangular search of objects:

df = SkyServer.rectangularSearch(min_ra=258.3, max_ra=258.31, min_dec=64,max_dec=64.01, dataRelease=SkyServer_DataRelease)
print(df)

In [ ]:
#do an object search based on RA,Dec coordinates:

object = SkyServer.objectSearch(ra=258.25, dec=64.05, dataRelease=SkyServer_DataRelease)
print(object)

# SciDrive section:

In [ ]:
#?SciDrive.publicUrl

In [ ]:
#define name of directory to be created in SciDrive:
SciDrive_Directory = "SciScriptR"
#define name, path and content of a file to be first created and then uploaded into SciDrive:
SciDrive_FileName = "TestFile.csv"
SciDrive_FilePath = "./TestFile.csv"
SciDrive_FileContent = "Column1,Column2\n4.5,5.5\n"

In [ ]:
#create a folder or container in SciDrive

responseCreate = SciDrive.createContainer(SciDrive_Directory)
print(responseCreate)

In [ ]:
#list content and metadata of directory in SciDrive

dirList = SciDrive.directoryList(SciDrive_Directory)
print(dirList)

In [ ]:
#get the public url to access the directory content in SciDrive

url = SciDrive.publicUrl(SciDrive_Directory)
print(url)

In [ ]:
#Delete folder or container in SciDrive:

responseDelete = SciDrive.delete(SciDrive_Directory)
print(responseDelete)

In [ ]:
#create a local file:

file = file(SciDrive_FileName)
writeLines(SciDrive_FileContent, file)
close(file)

In [ ]:
#uploading a file to SciDrive:

responseUpload = SciDrive.upload(path=paste(SciDrive_Directory, "/", SciDrive_FileName, sep=""), localFilePath=SciDrive_FilePath)
print(responseUpload)

In [ ]:
#download file:

fileContent = SciDrive.download(paste(SciDrive_Directory, "/", SciDrive_FileName, sep=""), format="text")
print(fileContent)

In [ ]:
#upload string data:

responseUpload = SciDrive.upload(path=paste(SciDrive_Directory, "/", SciDrive_FileName, sep=""), data=SciDrive_FileContent)
fileContent =  SciDrive.download(path=paste(SciDrive_Directory, "/", SciDrive_FileName, sep=""), format="text")
print(fileContent)

In [ ]:
#delete folder in SciDrive:

responseDelete = SciDrive.delete(SciDrive_Directory)
print(responseDelete)

In [ ]:
#delete local file:

file.remove(SciDrive_FilePath)

# SkyQuery section:

In [ ]:
#?SkyQuery.submitJob

In [ ]:
#list all databses or datasets available

datasets = SkyQuery.listAllDatasets()
print(datasets)

In [ ]:
#get info about the user's personal database or dataset

info = SkyQuery.getDatasetInfo("MyDB")
print(info)

In [ ]:
#list tables inside dataset

tables = SkyQuery.listDatasetTables("MyDB")
print(tables)

In [ ]:
#list available job queues

queueList = SkyQuery.listQueues()
print(queueList)

In [ ]:
#list available job queues and related info

quick = SkyQuery.getQueueInfo('quick')
long= SkyQuery.getQueueInfo('long')
print(quick)
print(long)

In [ ]:
#Define query

SkyQuery_Query = "select 4.5 as Column1, 5.5 as Column2"

In [ ]:
#submit a query as a job

jobId = SkyQuery.submitJob(query=SkyQuery_Query, queue="quick")
print(jobId)

In [ ]:
#get status of a submitted job

jobId = SkyQuery.submitJob(query=SkyQuery_Query, queue="quick")
jobDescription = SkyQuery.getJobStatus(jobId=jobId)
print(jobDescription)

In [ ]:
# wait for a job to be finished and then get the status

jobId = SkyQuery.submitJob(query=SkyQuery_Query, queue="quick")
jobDescription = SkyQuery.waitForJob(jobId=jobId, verbose=TRUE)
print("jobDescription=")
print(jobDescription)

In [ ]:
# cancel a job that is running, and then get its status

jobId = SkyQuery.submitJob(query=SkyQuery_Query, queue="long")
isCanceled = SkyQuery.cancelJob(jobId)
print(isCanceled)
print("job status:")
print(SkyQuery.getJobStatus(jobId=jobId))

In [ ]:
# get list of jobs

quickJobsList = SkyQuery.listJobs('quick')
longJobsList = SkyQuery.listJobs('long')
print(quickJobsList)
print(longJobsList)

In [ ]:
#define csv table to be uploaded to into MyDB in SkyQuery

SkyQuery_TestTableName = "TestTable_SciScript_R"
SkyQuery_TestTableCSV = "Column1,Column2\n4.5,5.5\n"

In [ ]:
#uploading the csv table:

result = SkyQuery.uploadTable(uploadData=SkyQuery_TestTableCSV, tableName=SkyQuery_TestTableName, datasetName="MyDB", format="csv")
print(result)

In [ ]:
#downloading table:

table = SkyQuery.getTable(tableName=SkyQuery_TestTableName, datasetName="MyDB", top=10)
print(table)

In [ ]:
#list tables inside dataset

tables = SkyQuery.listDatasetTables("MyDB")
print(tables)

In [ ]:
#get dataset table info:

info = SkyQuery.getTableInfo(tableName=paste("webuser.",SkyQuery_TestTableName, sep=""), datasetName="MyDB")
print(info)

In [ ]:
#get dataset table columns info

columns = SkyQuery.listTableColumns(tableName=paste("webuser.", SkyQuery_TestTableName, sep=""), datasetName="MyDB")
print(columns)

In [ ]:
#drop (or delete) table from dataset.

result = SkyQuery.dropTable(tableName=SkyQuery_TestTableName, datasetName="MyDB");
print(result)

# Files section:

In [ ]:
# defining the FileService name

Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName1 = "Storage"
Files_UserVolumeName1 = "UserVolume5"
Files_RootVolumeName2 = "Storage"
Files_UserVolumeName2 = "UserVolume9"
Files_NewDirectoryName1 = "myNewDirectory5"
Files_NewDirectoryName2 = "myNewDirectory9"
Files_LocalFileName = "MyNewFile.txt"
Files_LocalFileContent = "#ID,Column1,Column2\n1,4.5,5.5"

In [ ]:
token = Authentication.login(Authentication_loginName, Authentication_loginPassword);

In [ ]:
# get available FileServices

fileServices = Files.getFileServices();
fileServices

In [ ]:
# get names of file services available to the user.

fileServiceNames = Files.getFileServicesNames();
fileServiceNames

In [ ]:
# get FileService from Name

fileService = Files.getFileServiceFromName(Files_FileServiceName);
fileService

In [ ]:
# get the API endpoint URL of a FileService

fileServiceAPIUrl = Files.__getFileServiceAPIUrl(fileService);
fileServiceAPIUrl

In [ ]:
# get information about root volumes 

rootVolumes = Files.getRootVolumesInfo(fileService)
rootVolumes

In [ ]:
# get information about data volumes 

dataVolumes = Files.getDataVolumesInfo(fileService)
dataVolumes

In [ ]:
# get information about user volumes

userVolumes = Files.getUserVolumesInfo(fileService)
userVolumes

In [ ]:
Files.createUserVolume(fileService,paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName1,sep="/"))
Files.createUserVolume(fileService,paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName2,sep="/"))

In [ ]:
# create a directory in the user volumes

Files.createDir(fileService,paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName1,Files_NewDirectoryName1,sep="/"));
Files.createDir(fileService,paste(Files_RootVolumeName2,Authentication_loginName,Files_UserVolumeName2,Files_NewDirectoryName2,sep="/"));

In [ ]:
# upload a text string into a file in the remote directory

Files.upload(fileService, 
    paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName1,Files_NewDirectoryName1,Files_LocalFileName,sep="/"), 
             data=Files_LocalFileContent);

In [ ]:
# List content of remote directory

dirList = Files.dirList(fileService, 
    paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName1,Files_NewDirectoryName1,sep="/"),
    level=4)
dirList

In [ ]:
# download a remote text file into a local directory

Files.download(fileService, 
    paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName1,Files_NewDirectoryName1,Files_LocalFileName,sep="/") 
    ,localFilePath=Files_LocalFileName);

In [ ]:
# Delete remote file

Files.delete(fileService, 
paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName1,Files_NewDirectoryName1,Files_LocalFileName,sep="/"))

In [ ]:
# upload a local file into a remote directory

Files.upload(fileService, 
paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName1,Files_NewDirectoryName1,Files_LocalFileName,sep="/")
, localFilePath=Files_LocalFileName, quiet=FALSE);

In [ ]:
# copy remote file into remote directory

Files.move(fileService,
paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName1,Files_NewDirectoryName1,Files_LocalFileName,sep="/"), 
           fileService, 
paste(Files_RootVolumeName2,Authentication_loginName,Files_UserVolumeName2,Files_NewDirectoryName2,Files_LocalFileName,sep="/"));

In [ ]:
# sharing user volume with another user

Files.shareUserVolume(fileService, 
paste(Files_RootVolumeName2,Authentication_loginName,Files_UserVolumeName2,sep="/"), 
    sharedWith=Authentication_login_sharedWithName, sharee_type="USER",allowedActions=list("read"))

In [ ]:
# let the other user log-in

token1 = Authentication.login(Authentication_login_sharedWithName, Authentication_login_sharedWithPassword);

In [ ]:
# the other user downloads the remote text file (from the shared user volume) into a local string variable

string = Files.download(fileService, 
    paste(Files_RootVolumeName2,Authentication_loginName,Files_UserVolumeName2,Files_NewDirectoryName2,Files_LocalFileName,sep="/"), 
    format="txt");
print(string)

In [ ]:
# the original user logs in again

token1 = Authentication.login(Authentication_loginName, Authentication_loginPassword);

In [ ]:
# delete folders

Files.delete(fileService, 
    paste(Files_RootVolumeName1,Authentication_loginName,Files_UserVolumeName1,Files_NewDirectoryName1,sep="/"))
Files.delete(fileService, 
    paste(Files_RootVolumeName2,Authentication_loginName,Files_UserVolumeName2,Files_NewDirectoryName2,sep="/"))

In [ ]:
# delete user volumes

Files.deleteUserVolume(fileService, 
    paste(Files_RootVolumeName1, Authentication_loginName, Files_UserVolumeName1,sep="/"))
Files.deleteUserVolume(fileService, 
    paste(Files_RootVolumeName1, Authentication_loginName, Files_UserVolumeName2,sep="/"))

# Jobs section:

In [ ]:
Jobs_DockerComputeDomainName = 'Small Jobs Domain'

Jobs_FileServiceName = "FileServiceJHU"
Jobs_RootVolumeName = "Storage"
Jobs_UserVolumeName = "JobsTestVolume"
Jobs_DirectoryName = "JobsTestDirectory"
Jobs_NotebookName = 'TestNotebook.ipynb'
Jobs_ShellCommand = 'ls -laht'
Jobs_DockerImageName = 'Python + R'

Jobs_UserVolumes = list( list(name=Jobs_UserVolumeName, rootVolumeName=Jobs_RootVolumeName, owner=Authentication_loginName, needsWriteAccess=TRUE),
                         list(name="scratch", rootVolumeName="Temporary", owner=Authentication_loginName, needsWriteAccess=TRUE) ) 
Jobs_DataVolumes = list(list(name='SDSS DAS'))
Jobs_Parameters =  'param1=1\nparam2=2\nparam3=3'
Jobs_Alias = 'MyNewJob'

Jobs_SqlQuery = "select 1;"
Jobs_RBBComputeDomainName = 'Manga (long)'
Jobs_DatabaseContextName = "manga"
Jobs_QueryResultsFile = 'myQueryResults'

In [ ]:
# get docker compute domains

dockerComputeDomains = Jobs.getDockerComputeDomains();
print(dockerComputeDomains)

In [ ]:
# get names of docker compute domains available to the user

dockerComputeDomainsNames = Jobs.getDockerComputeDomainsNames()
print(dockerComputeDomainsNames)

In [ ]:
# get docker compute domain from name

dockerComputeDomain = Jobs.getDockerComputeDomainFromName(Jobs_DockerComputeDomainName)
print(dockerComputeDomain)

In [ ]:
# uploading Jupyter notebook to remote directory

fileService = Files.getFileServiceFromName(Jobs_FileServiceName);
Files.createUserVolume(fileService,paste(Jobs_RootVolumeName,Authentication_loginName,Jobs_UserVolumeName,sep="/"))
Files.upload(fileService, 
    paste(Jobs_RootVolumeName,Authentication_loginName,Jobs_UserVolumeName,Jobs_DirectoryName,Jobs_NotebookName,sep="/"), 
             localFilePath=Jobs_NotebookName);

In [ ]:
# submit a Jupyter notebook job.

jobId = Jobs.submitNotebookJob(notebookPath=paste('/home/idies/workspace',Jobs_RootVolumeName,Authentication_loginName,Jobs_UserVolumeName,Jobs_DirectoryName,Jobs_NotebookName,sep="/"),
                             dockerComputeDomain=dockerComputeDomain,
                             dockerImageName=Jobs_DockerImageName,
                             userVolumes=Jobs_UserVolumes,
                             dataVolumes=Jobs_DataVolumes,
                             resultsFolderPath="",
                             parameters=Jobs_Parameters,
                    jobAlias=Jobs_Alias)

print(jobId)

In [ ]:
# get job status

jobStatus = Jobs.getJobStatus(jobId)
print(jobStatus)

In [ ]:
# get job description

job = Jobs.getJobDescription(jobId)
print(job)

In [ ]:
# wait until job is finished and get job status

jobId = Jobs.submitNotebookJob(notebookPath=paste('/home/idies/workspace',Jobs_RootVolumeName,Authentication_loginName,Jobs_UserVolumeName,Jobs_DirectoryName,Jobs_NotebookName,sep="/"),
                             dockerComputeDomain=dockerComputeDomain,
                             dockerImageName=Jobs_DockerImageName,
                             userVolumes=Jobs_UserVolumes,
                             dataVolumes=Jobs_DataVolumes,
                             resultsFolderPath="",
                             parameters=Jobs_Parameters,
                             jobAlias=Jobs_Alias)

jobStatus = Jobs.waitForJob(jobId)
print(jobStatus)

In [ ]:
# cancel job

jobId = Jobs.submitNotebookJob(notebookPath=paste('/home/idies/workspace',Jobs_RootVolumeName,Authentication_loginName,Jobs_UserVolumeName,Jobs_DirectoryName,Jobs_NotebookName,sep="/"),
                             dockerComputeDomain=dockerComputeDomain,
                             dockerImageName=Jobs_DockerImageName,
                             userVolumes=Jobs_UserVolumes,
                             dataVolumes=Jobs_DataVolumes,
                             resultsFolderPath="",
                             parameters=Jobs_Parameters,
                    jobAlias=Jobs_Alias)

Jobs.cancelJob(jobId)

In [ ]:
Files.deleteUserVolume(fileService, 
                       paste(Jobs_RootVolumeName,Authentication_loginName,Jobs_UserVolumeName,sep="/"), 
                       quiet=TRUE)

In [ ]:
# submit shell command job

jobId = Jobs.submitShellCommandJob(shellCommand=Jobs_ShellCommand,
                                   dockerComputeDomain=dockerComputeDomain,
                                   dockerImageName=Jobs_DockerImageName,
                                   userVolumes=Jobs_UserVolumes,
                                   dataVolumes=Jobs_DataVolumes,
                                   resultsFolderPath="",
                                   jobAlias=Jobs_Alias)

In [ ]:
# get relational database (RDB) compute domains

rdbComputeDomains = Jobs.getRDBComputeDomains();
print(rdbComputeDomains)

In [ ]:
# get names of relational database (RDB) compute domains

rdbComputeDomainsNames = Jobs.getRDBComputeDomainsNames()
print(rdbComputeDomainsNames)

In [ ]:
# get relational database (RDB) compute domain from name

rdbComputeDomain = Jobs.getRDBComputeDomainFromName(Jobs_RBBComputeDomainName);
print(rdbComputeDomain)

In [ ]:
# submit RDB (relatinal database) query job

jobId = Jobs.submitRDBQueryJob(sqlQuery=Jobs_SqlQuery, 
                               rdbComputeDomain=rdbComputeDomain, 
                               databaseContextName = Jobs_DatabaseContextName, 
                               resultsName = Jobs_QueryResultsFile, 
                               resultsFolderPath = "",
                               jobAlias = Jobs_Alias)

print(jobId)

In [ ]:
# get job description

job = Jobs.getJobDescription(jobId)
print(job)

In [ ]:
# get list of jobs
jobs_list = Jobs.getJobsList(top=2)
jobs_list



In [ ]:
# get list of docker jobs
jobs_list = Jobs.getDockerJobsListQuick(top=2)
jobs_list